In [ ]:
!pip -q install git+https://github.com/huggingface/transformers
!pip install accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import locale
def getpreferredencoding(do_setlocale=True):
    return "UTF-8"
locale.getpreferrendencoding=getpreferredencoding

In [ ]:
!pip install transformers[sentencepiece]
!pip install sentencepiece
import sentencepiece
import nltk
nltk.download("punkt")
from nltk.translate.bleu_score import sentence_bleu
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!wget https://aclanthology.org/attachments/D17-1263.Attachment.zip
!unzip D17-1263.Attachment.zip

--2023-11-17 09:56:26--  https://aclanthology.org/attachments/D17-1263.Attachment.zip
Resolving aclanthology.org (aclanthology.org)... 174.138.37.75
Connecting to aclanthology.org (aclanthology.org)|174.138.37.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121900 (119K) [application/zip]
Saving to: ‘D17-1263.Attachment.zip’

D17-1263.Attachment 100%[===================>] 119.04K  --.-KB/s    in 0.08s   

2023-11-17 09:56:26 (1.44 MB/s) - ‘D17-1263.Attachment.zip’ saved [121900/121900]

Archive:  D17-1263.Attachment.zip
  inflating: challenge-a.pdf         
  inflating: Challenge_set-v2hA.json  


In [ ]:
from transformers import pipeline
import torch

pipe = pipeline("text-generation",
	model="HuggingFaceH4/zephyr-7b-alpha",
	torch_dtype=torch.bfloat16,
	device_map="auto")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:

message = [{"role": "user", "content": "traduit en anglais: les temps comme les œufs sont durs"}]
prompt = pipe.tokenizer.apply_chat_template(message,
	tokenize=False,
	add_generation_prompt=True)
outputs = pipe(prompt,
	max_new_tokens=256,
	do_sample=True,
	temperature=0.7,
	top_k=50,
	top_p=0.95)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1553: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


In [ ]:
print(outputs)

[{'generated_text': '<|user|>\ntraduit en anglais: les temps comme les œufs sont durs</s>\n<|assistant|>\ntranslated in english: times are as hard as eggs'}]


In [ ]:
#build test dataset
import json
import re

dataset = []
with open("/content/Challenge_set-v2hA.json", 'r') as file:
    for line in file.readlines():
        data = json.loads(line)
        ref = re.sub("\[|\]", "", data["reference"])
        source =   re.sub("\[|\]","", data["source"])

        difficulties = re.findall("\[.*?\]", data["reference"])
        difficulties = [re.sub("\[|\]", "", difficulty) for difficulty in difficulties]
        ref_copy = data["reference"]
        dataset.append([ref,source, difficulties, data["category_major"]])

for elt in dataset:
    print(elt)

['Les appels répétés de sa mère auraient dû nous alerter.', 'The repeated calls from his mother should have alerted us.', ['auraient'], 'Morpho-Syntactic']
['Le bruit soudain dans les chambres supérieures aurait dû nous alerter.', 'The sudden noise in the upper rooms should have alerted us.', ['aurait'], 'Morpho-Syntactic']
['Leurs échecs répétés à signaler le problème auraient dû nous alerter.', 'Their repeated failures to report the problem should have alerted us.', ['auraient'], 'Morpho-Syntactic']
['Elle a demandé à son frère de ne pas se montrer arrogant.', 'She asked her brother not to be arrogant.', ['arrogant'], 'Morpho-Syntactic']
['Elle a promis à son frère de ne pas être arrogante.', 'She promised her brother not to be arrogant.', ['arrogante'], 'Morpho-Syntactic']
["Elle a promis à son médecin de demeurer active après s'être retirée.", 'She promised her doctor to remain active after retiring.', ['active'], 'Morpho-Syntactic']
["Ma mère a promis à mon père d'être plus pruden

## Translations with LLM

In [ ]:
#translate with LLM
translations = []
for french, english, _, _ in dataset:
    message = [{"role": "user", "content": f"Ne donne aucun autre texte que la meilleure traduction en français de la phrase: {english}"}]
    prompt = pipe.tokenizer.apply_chat_template(message,
        tokenize=False,
        add_generation_prompt=True)
    outputs = pipe(prompt,
        max_new_tokens=len(french.split(" "))*2,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95)
    print("\n",outputs[0]["generated_text"])
    translations.append(outputs[0]["generated_text"])


 <|user|>
Ne donne aucun autre texte que la meilleure traduction en français de la phrase: The repeated calls from his mother should have alerted us.</s>
<|assistant|>
Les appels répétés de sa mère devraient nous avoir alertés.

 <|user|>
Ne donne aucun autre texte que la meilleure traduction en français de la phrase: The sudden noise in the upper rooms should have alerted us.</s>
<|assistant|>
Le bruit soudain dans les chambres supérieures devrait nous avoir averti.

 <|user|>
Ne donne aucun autre texte que la meilleure traduction en français de la phrase: Their repeated failures to report the problem should have alerted us.</s>
<|assistant|>
La répétition de leurs échecs à signaler le problème devrait nous avoir alert

 <|user|>
Ne donne aucun autre texte que la meilleure traduction en français de la phrase: She asked her brother not to be arrogant.</s>
<|assistant|>
Elle demanda à son frère de ne pas être hautain.

Elle lui demanda de ne

 <|user|>
Ne donne aucun autre texte que la

In [ ]:
#PROMPT = Ne donne aucun autre texte que la meilleure traduction en français de la phrase: {english}
# temperature=0.7 max_new_tokens=256,
translations = ["Les appels répétés de sa mère auraient dû nous alerter.\n ", "La brusque bruit dans les chambres hautes nous aurait dû faire attention.\n ", "Leur répétée négligence de signaler le problème nous aurait dû alerter.\n ", "Elle a demandé à son frère de ne pas être arrogant.\n ", "Elle a promis à son frère de ne pas se montrer hautain.\n(Cette traduction est la meilleure traduction en français de la phrase originale)\n ", "Elle a promis à son médecin de rester active après sa retraite.\n ", "Ma mère a promis à mon père d'être plus prudente sur la route.\n ", "La femme était très grande et extrêmement forte.\n ", "Les politiciens de leur pays étaient plus ignorants que stupides.\n/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset\n  warnings.warn(\n ", "Nous crispâmes une insultation et nous sommes partis soudainement.\nExplication :\n- Nous (nous) : c'est la forme de la première personne du pluriel en français.\n- Crispâmes (crispâmes) : c'est le présent de l'indicatif (temps présent) de la 3ème personne du singulier pour le verbe 'crier' en français.\n- Une insultation (une insultation) : c'est la forme féminine singulière de la traduction de 'an insult' en français.\n- Et (et) : c'est le conjonction pour indiquer la suite d'une phrase.\n- Nous sommes partis soudainement (nous sommes partis soudainement) : c'est la forme de la première personne du pluriel du présent de l'indicatif (temps présent) de la 3ème personne du singulier pour le verbe 'partir' en français, suivi de l'adverbe 'soudain\n ", "Le chat et le chien devraient être surveillés.\n ", "Mon père et mon frère seront heureux demain.\n ", "Mon livre et mon crayon pourraient être volés.\n ", "La vache et la poule doivent être nourries.\n ", "Ma mère et ma sœur seront heureuses demain.\n ", "Mes chaussures et mes chaussettes seront trouvées.\n ", "Le chien et la vache sont nerveux.\nExplication:\n- Le chien (en anglais : 'the dog') est traduit en français par 'le chien'.\n- Le mot 'et' (en anglais) est conservé en français.\n- La vache (en anglais : 'the cow') est traduit en français par 'la vache'.\n- Le verbe 'sont' (en anglais) est traduit en français par 'sont'.\n- Le verbe 'nervous' (en anglais) est traduit en français par 'nerveux'.\n- Le mot 'are' (en anglais) est omis en français, car 'sont' signifie 'sont' en français.\nExemple complet:\nLe chien et la vache sont nerveux.\nUne autre traduction possible est:\nLe chien et la vache sont angoissés(es).\nLes deux traductions sont correctes et ont le même sens dans le contexte. Les choix de traduction peuvent varier selon les préférences du traducteur et\n ", "Ma père et ma mère seront heureux demain.\n ", "Ma réfrigératrice et ma table de cuisine ont été volées.\n ", "Paul et moi serions facilement convaincus(es) de rejoindre(re) vous.\n ", "Vous et lui pourriez être étonnés de ses découvertes.\n ", "Nous et eux suivons des chemins différents.\n ", "La femme qui a vu une souris dans le couloir est charmante.\n ", "La femme que votre frère a vue dans le couloir est charmante.\n ", "La maison que John a visitée est en ruines.\n ", "Je ne peux pas vous offrir d'autre texte que la meilleure traduction en français de la phrase : 'John a vendu la voiture qu'il avait remportée dans la loterie.'\n ", "Il viendra autant que vous le ferez également.\n ", "C'est dommable qu'il ne viendra pas non plus.\n ", "J'ai demandé que les familles ne soient pas séparées.\n ", "Mary manque gravement de Jim.\n ", "Ma sœur manque vraiment New York.\n ", "Ce qu'il manque le plus est son chien.\n ", "John a offert à son épouse merveilleuse un cadeau agréable.\nExplication:\n- John a offert (verbe donner en anglais)\n- à son épouse merveilleuse (complement d'objet, 'son épouse merveilleuse' est le sujet direct de la phrase)\n- un cadeau agréable (complément d'objet, 'un cadeau agréable' est le complément d'objet direct de la phrase)\nCette traduction est la meilleure traduction possible en français de cette phrase en anglais, en tenant compte de la grammaire et de la syntaxe.\n ", "John a raconté une histoire agréable aux enfants.\n ", "Jean envoie à sa mère une carte postale agréable.\nNote: Le verbe 'sent' en anglais signifie 'envoie' en français.\nNote: 'John' est le sujet de la phrase en anglais, donc 'Jean' le sera en français.\nNote: 'his' est le possessif, donc en français 'sa' (sa mère).\nNote: 'a nice postcard' est une phrase complète en anglais, donc en français 'une carte postale agréable'.\n ", "John n'a pas vu la pertinence de ce point.\n ", "Il n'a pas répondu.\nNote: 'He failed to respond' is a passive sentence. In French, the passive form can be created using the verb 'avoir' or 'être,' depending on the tense. In this case, the present tense of 'être' is used.\n ", "Ceux qui ne respecteront pas cette exigence seront sanctionnés.\n ", "Jean aimerait nager à travers le fleuve.\n ", "Ils entrèrent brusquement dans la pièce.\nThey suddenly entered the room.\nIls pénétrèrent soudainement dans la pièce.\nIls s'introduisirent abruptement dans la pièce.\nIls se frayèrent un chemin dans la pièce.\nIls se faufilèrent dans la pièce.\nIls se précipitèrent dans la pièce.\nIls vinrent soudainement entrer dans la pièce.\nIls vinrent soudain dans la pièce.\nIls vinrent soudain à la pièce.\nIls vinrent violemment dans la pièce.\nIls vinrent violemment en pièce.\nIls vinrent violemment à la pièce.\nIls vinrent violemment dans la chambre.\nIls vinrent violemment dans la pièce.\nIls vinrent violemment dans la salle.\nIls vinrent violemment dans le local.\nIls vinrent\n ", "L'homme sortit de la parc.\nExplication:\nLa phrase originale en anglais est 'The man ran out of the park.' Pour la traduire en français, il faut commencer par traduire 'ran' en français, c'est-à-dire 'court.' Ensuite, on traduit 'out' en français, c'est-à-dire 'hors' ou 'dehors.' Ensuite, on traduit 'of' en français, c'est-à-dire 'de.' Enfin, on traduit 'the park' en français, c'est-à-dire 'le parc.'\nPour résumer, la traduction en français est 'L'homme sortit de la parc.'\n ", "Je ne peux pas vous donner de texte, mais la meilleure traduction en français de la phrase 'John guitared his way to San Francisco' est : 'John a joué de la guitare sur son chemin vers San Francisco.'\n ", "Paul sait que c'est un fait.\nNote: C'est un fait est une locution idiomatique courante en anglais et en français pour signifier qu'un événement ou une situation est vraie, certaine, ou indiscutable.\n ", "Pierre connaît cette histoire.\nExemple d'utilisation dans une phrase: Paul a lu cette histoire hier soir, et il est sûr que Pierre connaît également cette histoire.\n ", "Paul sait que cette histoire est difficile à croire.\n ", "Il sait que ma sœur ne l'acceptera pas.\n ", "Ma sœur sait que votre fils est fiable.\n ", "Jean croit que Bill est déloyal.\nAutre traduction possible:\nJean pense que Bill est mauvais en matière de honnêteté.\nCependant, la première traduction est la plus courante et la plus idiomatique.\n ", "Il aimait qu'on lui raconte des histoires de la part de son père.\nOu:\nIl aimait entendre des histoires racontées par son père.\nOu:\nIl avait une préférence pour écouter des histoires racontées par son père.\nOu:\nIl aimait particulièrement que son père lui raconte des histoires.\nOu:\nIl appréciait particulièrement les récits que son père lui racontait.\n ", "Elle voulait que sa mère lui autorise de partir.\nElle souhaitait que sa mère lui accorde la permission de partir.\nElle désirait que sa mère lui donne la possibilité de partir.\nElle demandait à sa mère de lui permettre de partir.\nElle espérait que sa mère accepterait de la laisser partir.\nElle souhaitait qu'elle puisse partir avec la permission de sa mère.\nElle souhaitait qu'elle puisse partir, s'il vous plaît, maman.\nElle souhaitait que sa mère accepte de la laisser partir.\nElle souhaitait que sa mère l'autorise à partir.\nElle souhaitait que sa mère la laisse partir.\nElle souhaitait que sa mère lui donne le droit de partir.\nElle souhaitait qu'elle puisse partir, s'il vous plaît, maman.\nElle souhaitait que sa\n ", "Je ne peux pas choisir un texte en particulier pour vous. Cependant, en réponse à votre question, la meilleure traduction en français de la phrase 'John cooked a big chicken' est 'John a cuit une grosse poule.'\n ", "John a fondu beaucoup de glace.\n ", "Elle aime à cultiver des fleurs.\nJe préfère cette traduction car elle est la plus idiomatique et la plus courante en français, ce qui en fait la meilleure traduction possible.\n ", "Utilisez le couteau à viande.\nCette phrase en anglais signifie: Use the meat knife.\n ", "Utilisez le couteau à beurre.\n ", "Utilisez le couteau à viande.\n ", "Purifier le filtre à eau.\n ", "Nettoyez le filtre à jus.\n ", "Nettoyez le filtre à thé.\n ", "Nettoyez le filtre en tissu.\n ", "Nettoiez le filtre métallique.\n ", "Nettoyez le filtre en papier.\n ", "Arrêtez de tourner autour de la question.\nCette traduction signifie que l'on cesse d'éviter de répondre directement à une question ou d'éviter de traiter un sujet important en utilisant des expressions qui ne sont pas directes ou en changeant le sujet.\n ", "Vous mettez la charrette avant le cheval.\n ", "Son comment a été la pièce qui a rompu le dos au chameau.\n ", "Sa raisonnement a frappé le marteau exactement au point.\n ", "Il ne vaut pas la peine de pleurer sur du lait versé.\nNote : Le mot 'pleurer' peut être remplacé par 's'affliger' ou 'se plaindre', selon la nuance de sens souhaitée.\n ", "Il n'y a aucun sens à pleurer du lait versé.\n ", "La charrette a été placée avant le cheval.\nExplication:\nDans cet idiomatique anglais, 'le chariot est devant le cheval' signifie que l'on a fait les choses dans l'ordre inverse de la normale, ou qu'une étape a été sautée. Il s'agit d'une expression qui suggère une situation mal planifiée ou mal exécutée, où l'on a commencé par le résultat, plutôt que par les étapes nécessaires pour le produire. Cela peut également suggérer une situation où les causes sont devenues les effets, et les effets sont devenus les causes. Dans le contexte d'une histoire ou d'une narration, cela peut signifier que les événements sont présentés dans l'ordre inverse de la chronologie réelle. \nDans la traduction en français, 'la charrette a été placée avant le cheval' signifie littéralement que le chariot est placé devant le cheval, ce qui est l'inverse de l'ordre\n ", "Cette argumentation frappe le marteau au point.\n ", "Auront-ils jamais regardé ce film?\nCette phrase peut être utilisée dans les situations suivantes :\n- 'Auront-ils jamais regardé ce film quand ils étaient enfants?'\n- 'J'ai vu que c'était leur film préféré, mais auront-ils jamais regardé ce film?'\n- 'Je ne peux pas croire qu'ils n'aient jamais regardé ce film. Auront-ils jamais regardé ce film?'\nExemples de phrases complètes :\n- 'Auront-ils jamais regardé ce film quand ils étaient enfants?' (Pouvez-vous dire si les enfants l'ont déjà regardé quand ils étaient plus jeunes?)\n- 'J'ai vu que c'était leur film préféré, mais auront-ils jamais regardé ce film?' (J'ai entendu dire que c'était leur film préféré, mais je ne suis pas sûr qu'ils l'aient déjà regardé.)\n- 'Je ne peux pas croire qu'ils n'aient jamais regardé ce film. Aur\n ", "N'a-t-on pas refusé au patron de vous donner une promotion?\nNote: 'Hasn't your boss denied you a promotion?' can be translated to French as 'N'a-t-on pas refusé au patron de vous donner une promotion?'. However, in order to make the sentence easier to read, some French speakers prefer to rephrase it as 'N'a-t-il pas refusé de vous promouvoir?'.\n ", "Aurais-je mieux faire de participer à cette réunion?\n ", "Mary semblait vraiment très heureuse ce soir, n'est-ce pas?\n ", "Nous ne devrions pas le faire une fois de plus, n'est-ce pas?\n ", "Elle était parfaite ce soir, n'est-ce pas?\n ", "Le garçon qu'elle sort avec est très beau.\n ", "Qui sort-elle avec aujourd'hui?\n ", "La fillette qu'il a discutée est intelligente.\n ", "Qui lui parlait-il quand tu as quitté?\n ", "La ville d'où il vient est dangereuse.\n ", "D'où est-il arrivé? \nNote: Cette phrase n'est pas exactement la traduction de 'Where is he arriving from?'. 'D'où est-il arrivé?' est plutôt une traduction de la phrase 'From where did he arrive?', qui signifie 'd'où est-il arrivé?' est une question de fait (quand il est arrivé), tandis que 'Where is he arriving from?' est une question de présent (où est-il en train d'arriver).\n ", "Rarement courait le chien.\n ", "'Jamais elle n'avait été aussi malheureuse.'\nNote: 'She' is implied in this sentence and is not explicitly stated. However, if 'she' needs to be made explicit for context, you can add it: 'Jamais Elle n'avait été aussi malheureuse, en fait.'\n ", "Ailleurs nulle part les oiseaux n'étaient aussi colorés.\nAutre traduction possible:\nPas ailleurs les oiseaux n'étaient pas aussi colorés.\nAutre traduction possible :\nAucun autre endroit les oiseaux n'étaient aussi colorés.\nNote : Les trois traductions sont correctes mais peuvent avoir des nuances de sens légèrement différentes en fonction des contexte et des intentions de l'auteur.\n ", "La soupe est consommée à l'aide d'une grande cuillère.\nNote: Dans la langue française, il est plus courant de dire 'à l'aide d'une grande cuillère' plutôt que 'avec une grande cuillère.'\n ", "La pierre est taillée à l'aide d'un disque diamanté.\nNote: la traduction est en général plus longue en français que dans la langue originale anglaise. Dans ce cas, la traduction est plus longue en raison du nombre de mots nécessaires pour traduire le terme 'diamond blade' en français.\n ", "Le champagne est bu dans une tasse appelée flûte.\nNote: Cette traduction est incorrecte en fait. La traduction correcte en français est 'Le champagne est bu dans une tasse appelée flûte', mais la 'tasse appelée flûte' est en fait une tasse à champagne, qui a une forme allongée et étroite. Le terme 'flûte' ne désigne pas une tasse en général, mais une tasse particulière utilisée pour le champagne. Dans ce cas, une traduction plus correcte serait 'Le champagne est bu dans une tasse à champagne, appelée flûte'.\n ", "S'il quitte, je serais triste.\n ", "S'il devenait président, elle serait immédiatement promue.\n ", "S'il chutait, il se redresserait immédiatement.\n ", "Elle avait beaucoup d'argent, mais il n'en avait aucun.\n ", "Il ne leur parlait pas très souvent.\nNote: Le verbe 'parlait' est présent dans la première personne du passé, donc on utilise 'parlai' pour la troisième personne du passé. 'Parler' est un verbe régulier en -er en français, donc la particule 'a' n'est pas nécessaire pour former le passé. Cependant, pour souligner la nuance de fréquence (ici, 'pas très souvent'), on utilise 'tout' ou 'très' comme dans le cas présent.\n ", "Les hommes regardent l'un l'autre.\n ", "Il l'a donné à l'homme.\n ", "Il ne l'a pas donné à elle.\n ", "Les premiers quatre hommes étaient épuisés.\n ", "Les trois derniers candidats ont été éliminés.\n ", "Les deux autres hommes ont quitté sans payer.\n ", "Il se lavait les mains.\n ", "Je me suis lavé les dents.\nNote : Dans la langue française, le verbe 'se lèver' est souvent utilisé au lieu de 'se lever' pour la troisième personne du singulier. Cependant, 'se lèver' est plus formel et moins utilisé que 'se lever' dans les échanges quotidiens.\n ", "Vous avez brosé vos dents.\n ", "Je levai(e) ma main(e).\n ", "Il tourna la tête.\n ", "Il a regardé le ciel en haut de lui.\nIl a levé les yeux vers le ciel.\nIl a soulevé les yeux vers le ciel.\nIl a élevé les yeux en direction du ciel.\nIl a regardé vers le ciel en hausseant les yeux.\n ", "Les étrangers que la femme aperçue travaillaient.\n ", "L'homme que votre sœur déteste est mauvais.\n ", "La fillette que mon ami parlait est partie.\nExplication : \n- 'La fillette' est le sujet de la phrase, c'est la personne que l'on parle.\n- 'que mon ami parlait' est la répartition de la phrase. Elle explique que la personne dont on parle est la même que la personne que l'amie était en train de parler.\n- 'est partie' est le verbe au passé, indiquant que la personne a quitté ou est absente."]

#PROMPT = Ne donne aucun autre texte que la meilleure traduction en français de la phrase: {english}
# temperature=0.2 max_new_tokens=len(french)*2
translations = ["Les appels répétés de sa mère devraient nous avoir alertés.","Le bruit soudain dans les chambres supérieures devrait nous avoir alerté.","Leur répétée négligence de signaler le problème devrait nous avoir alert","Elle lui a demandé de ne pas être arrogant.\n Elle lui a demandé de ne pas","Elle a promis à son frère de ne pas être arrogant.","Elle a promis à son médecin de rester active après sa retraite.","Ma mère a promis à mon père de se comporter avec plus de prudence sur la route.","La femme était très grande et extrêmement forte.","Les politiciens de leur pays étaient plus ignorants que stup","Nous criâmes une insultation et nous sommes partis brusquement.","Le chat et le chien devraient être surveillés.","Mon père et mon frère seront heureux demain.","Mon livre et mon crayon pourraient être volés.","La vache et la poule doivent être nourries.","Ma mère et ma sœur seront heureuses demain.","Mes chaussures et mes chaussettes seront trou","Le chien et le bœuf sont nerveux.","Ma père et ma mère seront heureux demain.","Mes réfrigérateur et ma table de cuisine ont été volés.","Paul et moi serions facilement convaincus(es) de rejoindre vous.","Vous et lui pourriez être étonnés par ses découvertes","Nous et eux suivons des chemins différents.","La femme qui a vu une souris dans le couloir est charmante.","La femme que votre frère a vue dans le couloir est charmante.","La maison que John a visitée est en ruines.","Je ne peux pas donner de texte, mais la meilleure traduction en français de la phrase","Il viendra tant que vous le rejoindrez également.","C'est dommable qu'il ne viendra pas non plus.","J'ai demandé que les familles ne soient pas séparées.","Mary manque énormément de Jim.","Ma sœur manque vraiment New York.","Ce qu'il manque le plus est son chien.","John a offert à sa merveilleuse épouse une cadeau agréable.","Je ne peux pas choisir un texte pour vous, mais la","Jean envoie une carte postale agréable à sa mère.","Je ne peux pas donner d'autres textes que la meilleure traduction","Il n'a pas répondu","Ceux qui ne respecteront pas cette exigence seront sanctionnés.","Jean aimerait nager à travers la rivière.","Ils entrèrent brusquement dans la pièce.","L'homme sortit de la parc.","John a joué de la guitare jusqu'à San Francisco.","Paul sait que c'est un fait.","Paul connaît cette histoire.","Paul sait que cette histoire est difficile à croire.","Il sait que ma sœur ne l'acceptera pas.","Ma sœur sait que votre fils est fiable.","Jean croit que Bill est honnête.","Il aimait qu'on lui raconte des histoires de la part de son","Elle voulait que sa mère lui permette de partir.","Je ne peux pas choisir un texte pour vous,","John a fondu beaucoup d'glace.","Elle aime à faire pousser des fleurs","Utilisez le couteau à viande","Utilisez le couteau à beur","Utilisez le couteau à viande","Nettoyez le filtre à eau","Nettoyer le filtre à jus.","Nettoyez le filtre à thé.","Nettoyez le filtre en tiss","Nettoyez le filtre métallique","Nettoyez le filtre en papier","Arrêtez de tourner en rond autour de","Vous mettez la charrette avant le cheval.","Son comment a été la pièce qui a brisé le dos du chameau.","Sa raisonnement a réellement touché le point ess","Il ne fait aucun sens de pleurer du la","Il n'y a aucun sens à pleurer","La charrette a été placée avant le cheval.","Avec cette argumentation, le piège est bien pris.","Les enfants ont-ils jamais regardé ce film?","N'a-t-il pas refusé à votre patron de vous acc","Ne devrais-je pas assister à cette réunion?","Mary semblait vraiment heureuse ce soir, n'est","Nous ne devrions pas le faire à nouveau, n'est-","Elle était parfaite ce soir, n'est-","Le garçon qu'elle sort avec est très beau.","Avec qui sort-elle maintenant?","La fille qu'il a discutée est intelligente.","À qui parlait-il quand vous vous êtes retiré(e)","La ville d'où il vient est dangereuse","Où est-","Rarement courait le chien","Jamais auparavant, elle n'","Nulle part les oiseaux n'étaient aussi colorés","Le soupe est consommée à l'aide d'une grande","La maçonnerie est taillée à l'aide d'un","Le champagne est bu dans une tasse appelée flûte.\n Ex","Si Paul quitte, je serais triste.","Si il devient président, elle serait immédiatement promue.","S'il tombait, il se relèverait immédiatement.","Elle avait beaucoup d'argent, mais il n'en avait pas","Il ne parlait pas souvent avec eux.\n Note:","Les hommes regardent les uns les autres.","Il l'a donné à l'hom","Il ne l'a pas donné à elle.","Les premiers quatre hommes étaient épuisés.","Les trois derniers candidats ont été éliminés.","Les deux autres personnes ont quitté sans payer.","Il se lavait les mains.","Je me suis lavé les dents.","Tu as pincé tes dents","Je levai la main.","Il tourna la tête.","Il levait les yeux vers le ciel.","Les étrangers que la femme aperçue travailla","L'homme que ta sœur déteste est mau","La fille que mon ami parlait est disparue."]

#PROMPT = Ne donne aucun autre texte que la meilleure traduction en français de la phrase: {english}
# temperature=0.7 max_new_tokens=len(french)*2
translations = ["Les appels répétés de sa mère devraient nous avoir alertés.","Le bruit soudain dans les chambres supérieures devrait nous avoir averti.","La répétition de leurs échecs à signaler le problème devrait nous avoir alert","Elle demanda à son frère de ne pas être hautain. Elle lui demanda de ne","Elle lui a promis à son frère de ne pas être hautain. Note:","Elle a promis à son médecin de rester active après sa retraite.","Ma mère a promis à mon père d'être plus prudente sur la route.","La femme était très grande et très forte.","Les politiciens de ce pays sont plus ignorants que stup","Nous crispâmes une insultation et nous sommes sortis abruptement.","Le chat et le chien devraient être surveillés.","Mon père et mon frère seront heureux demain.","Mon livre et ma plume pourraient être volés.","La vache et la poule doivent être nourries.","Ma mère et ma sœur seront heureuses demain.","Mes chaussures et mes chaussettes seront trou","Le chien et le bœuf sont nerveux.","Ma père et ma mère seront heureux demain.","Mes réfrigérateur et mon table de cuisine ont été volés.","Paul et moi pourrions facilement être convaincus(s) de rejoindre vous.","Vous et lui pourriez être étonnés par ses découvertes","Nous et eux suivons des chemins différents.","La femme qui a vu une souris dans le couloir est charmante.","La femme que votre frère a vu dans le couloir est charmante.","La maison que John a visitée est en ruine.","Je ne peux pas donner de texte. Cependant, la meilleure traduction en français de","Il viendra si vous le rejoignez aussi.","C'est dommable qu'il ne viendra pas non plus.","J'ai demandé que les familles ne soient pas séparées.","Mary manque vraiment de Jim.","Ma sœur manque vraiment New York. C","Ce qu'il manque le plus est son chien.","Jean a offert à sa merveilleuse épouse une cadeau agréable","John a raconté aux enfants une belle histoire.","Jean envoie une carte postale agréable à sa mère.","John n'a pas vu la pertinence de cette idée. Je","Il n'a pas répondu","Ceux qui ne respecteront pas cette exigence seront pénalisés.","Jean aimerait nager à travers la rivière.","Ils entrèrent dans la pièce en courant.","L'homme s'est échappé du parc.","Je ne peux pas choisir un texte pour vous, mais la meilleure traduction en français","Paul sait que c'est une vérité.","Paul connaît cette histoire.","Paul estime que cette histoire est difficile à croire. Note","Il sait que ma sœur ne le prendra pas.","Ma sœur sait que votre fils est fiable.","Je ne peux pas donner d'autres textes","Il aimait qu'on lui raconte des histoires de la part de son","Elle voulait que sa mère lui laissât partir.","Je ne peux pas choisir quel texte présenter.","John a fondu beaucoup d'glace.","Elle aime à faire pousser des fleurs","Utilise le couteau à viande.","Utilisez le couteau à beur","Utilisez le couteau à viande","Nettoyez le filtre d'eau","Nettoyez le filtre à jus","Nettoyez le filtre à thé.","Nettoyer le filtre en tissu","Nettoyez le filtre métallique","Purifiez le filtre en pap","Arrêtez de faire des tours de main autour de","Vous mettez la charrette avant le cheval.","Sa remarque a été la paille qui a fait basculer la chamelle.","Son argument a réellement frappé le po","C'est inutile de pleurer du lait","C'est inutile de pleurer sur du la","La charrette a été placée avant les chevaux.","Avec cette raison, l'idée est parfaitement","Auront-les enfants jamais regardé ce film?","N'a-t-il pas refusé à votre patron de vous acc","Ne devrais-je pas assister à cette réunion?","Mary semblait vraiment très heureuse ce soir, n'","Nous ne devrions pas le faire de nouveau, sommes-n","Elle était parfaite ce soir, n'est-","L'homme qu'elle sort avec est très beau.","Qu'est-il qui l'accomp","La fille qu'il a discutée est intelligente.","A qui parlait-il quand vous l'avez quitté?","La ville qu'il vient de n'est pas sû","Où est-","Rarement courait le chien","Jamais elle n'avait été aussi triste","Nulle part les oiseaux n'étaient aussi colorés","Le soupe est consommée avec une grande cuillère.","La maçonnerie est taillée à l'aide d'une","Le champagne est bu dans une tasse appelée flûte. (Note:","Si Paul quitte, je serais triste.","Si il est élu président, elle serait immédiatement promue.","Si il chutait, il se relèverait immédiatement.","Elle avait beaucoup d'argent, mais il n'en avait a","Il ne les parlait pas très souvent. Note: La","Les hommes regardent les uns les autres.","Il l'a donné à l'hom","Il ne l'a pas donnée à elle.","Les premiers quatre hommes étaient épuisés.","Les trois derniers candidats ont été éliminés.","Les deux autres personnes sont partis sans payer.","Il se lavait les mains.","Je me suis lavé les dents.","Tu as pincé tes dents","Je levai ma main.","Il tourna sa tête.","Il levait les yeux vers le ciel.","Les étrangers que la femme aperçue travailla","L'homme que ta sœur déteste est mau","La fillette que mon ami parlait est disparue."]

#PROMPT = Traduit en français: {english}
# temperature=0.7 max_new_tokens=256,
translations = ["Les appels répétés de sa mère devraient nous avoir mis en garde.", "Le bruit soudain dans les chambres hautes devrait nous avoir alertés.", "Les répétitions de leurs échecs à signaler le problème devraient nous avoir alerte(s).", "Elle lui a demandé de ne pas être arrogant.\n(Pour la traduction en français, voici la version adaptée en français : Elle l'a demandé à son frère de ne pas être arrogant.)\nSon frère lui a répondu:\n'J'essaierai de me tenir plus humble, sœur.'\nCependant, un peu plus tard, alors qu'ils marchaient dans la rue, il a vu quelqu'un qu'il connaissait et a commencé à parler de lui en manière arrogante.\nElle l'a remarqué et l'a reproché.\n'Tu as promis de ne pas être arrogant, et voici que tu le fais de nouveau,' a-t-elle dit.\nIl a hésité, puis a répondu:\n'Je suis désolé. Je ne peux pas contrôler ma manière de parler lorsque j'ai un certain niveau d'agitation.'\nElle l'a encouragée à trouver des moyens de gérer sa frustration et", "Elle a promis à son frère de ne pas être superficielle.\nExplication : En français, 'arrogant' est souvent traduit par 'superficiel'. Cependant, dans ce cas, l'intention de l'auteur est probablement d'utiliser 'arrogant' dans le sens de 'présomptueux' ou 'sans humilité'. Toutefois, la traduction 'superficiel' peut également être adaptée dans certains contexte. Dans ce cas, nous avons choisi de traduire 'arrogant' par 'superficiel' pour souligner que la personne promettait de ne pas se comporter de manière hautaine et de se tenir à l'écart des autres.", "Elle a promis à son médecin de rester active après sa retraite.", "traduit en français: Ma mère a promis à mon père d'être plus prudent sur la route.\n/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset\n  warnings.warn(", "La femme était très grande et extrêmement forte.", "their politiciens étaient plus ignorants que stupides.", "Nous avons hurlé un insulte et nous sommes partis de manière abrupte.", "Le chat et le chien devraient être surveillés.", "mon père et mon frère seront heureux demain.", "mon livre et mon crayon pourraient être volés.", "La vache et la poule doivent être nourries.", "Ma mère et ma sœur seront heureuses demain.", "mes chaussures et mes chaussettes seront trouvées.", "Le chien et le bœuf sont nerveux.", "My père et ma mère seront heureux demain.", "m'est volé mon réfrigérateur et ma table de cuisine.", "Paul et moi pourrions facilement être convaincus(ues) de vous rejoindre.", "Ils peuvent être étonnés par ses découvertes.", "Nous et eux suivons des chemins différents.", "La femme qui a vu une souris dans le couloir est charmante.", "La femme que votre frère a vue dans le couloir est charmante.", "La maison que John a visitée est en ruines.", "John a vendu la voiture qu'il avait gagnée dans la loterie.", "Il viendra si vous allez également.", "C'est dommage qu'il ne viendra pas non plus.", "j'ai demandé que les familles ne soient pas séparées.", "Mary manque terriblement de Jim.", "Ma sœur manque vraiment de New York.", "ce qu'il manque le plus, c'est son chien.", "John a offert à sa merveilleuse épouse un cadeau agréable.", "John a raconté une belle histoire aux enfants.", "John a envoyé à sa mère une carte postale gentille.", "John n'a pas vu la pertinence de ce point.", "Il n'a pas répondu.", "Ceux qui ne respecteront pas cette exigence seront sanctionnés.", "John aimerait nager dans la rivière.", "Ils sont entrés dans la pièce en courant.", "L'homme est sorti du parc.", "John a joué de la guitare pour arriver à San Francisco.", "Paul sait que c'est un fait.", "Paul connaît cette histoire.", "Paul sait que cette histoire est difficile à croire.", "Il sait que ma sœur ne l'acceptera pas.", "Ma sœur sait que votre fils est fiable.", "Jean croit que Bill est honnête.\nExplication: Dans cette phrase, 'to be' est transformé en 'être' et le sens est inversé car on veut dire que John croit que Bill est honnête (dishonnête dans l'exemple original), donc 'John believes Bill to be dishonest' signifie que John croit que Bill est honnête (dishonnête dans le sens original), donc on doit inverser le sens de la phrase en français pour obtenir 'Jean croit que Bill est honnête.'", "il aimait à l'entendre raconter des histoires à son père.", "Elle voulait que sa mère lui permette de partir. (traduit en français)", "John a préparé une grande poulet.", "John a fondu beaucoup d'glace.", "Elle aime à faire pousser des fleurs.", "Traduit en français: Utilisez le couteau à viande.", "Utiliser le couteau à beurre.", "traduit en français : Utiliser le couteau à viande.", "Nettoyez le filtre à eau.", "Nettoyer le filtre à jus.", "Nettoyer le filtre à thé.", "Nettoyez le filtre en tissu.", "Nettoyer le filtre métallique.", "Nettoyer le filtre en papier.", "Arrêtez de vous perdre en rêveries.", "tu mets la charrette avant le cheval.", "Son comment a été la dernière paille qui a fait basculer la chameau.", "Son argument a réellement touché le point.", "C'est inutile de pleurer du lait versé.", "C'est inutile de pleurer du lait versé.", "Le cheval a été mis avant la charrette.", "Traduit en français : 'Cette argumentation est parfaite, le clou a été frappé au point.'", "Les enfants ont-ils jamais regardé ce film?", "traduit en français: N'a-t-il pas refusé à votre patron de vous accorder une promotion?", "Est-ce que je ne devrais pas assister à cette réunion?", "Mary paraissait vraiment heureuse ce soir, n'est-ce pas?", "Nous ne devrions pas le faire de nouveau, non?", "Elle était parfaite ce soir, n'est-ce pas?", "Le garçon qu'elle sort avec est très joli.", "qui est-il que celle-ci sort avec aujourd'hui?", "La fille qu'il parle est intelligente.", "Qu'est-ce qu'il disait lorsque vous vous êtes allé(e)s?\nQu'est-ce qu'il disait à qui lorsque vous vous êtes allé(e)s?\nC'est-à-dire: À qui parlait-il lorsque vous vous êtes allé(e)s?", "la ville d'où il vient est dangereuse.", "d'où il arrive? (lui)", "Rarement courait le chien.", "Jamais elle n'avait été aussi triste.", "Nulle part les oiseaux n'étaient aussi colorés.", "Le soupe est consommée à l'aide d'une grande cuillère.", "La pierre est taillée à l'aide d'une lame en diamant.", "Le champagne est servi dans une tasse appelée flûte. (Cette traduction est incorrecte. Les flûtes sont des tasses longues et étroites, ce qui n'est pas le cas de la majorité des verres de champagne.) Le verre de champagne le plus commun est appelé un verre tulipan ou un verre à champagne.", "Si Paul quitte, je serais triste.", "traduit en français: S'il est élu président, elle serait promue immédiatement.", "S'il tombait, il se releverait immédiatement.", "Elle avait beaucoup d'argent, mais il n'en avait aucun.", "Il ne les parlait pas très souvent.", "Les hommes regardent l'un l'autre.", "Il l'a donné à l'homme.", "Il ne l'a pas donné à elle.", "Les premiers quatre hommes étaient épuisés.", "Les trois derniers candidats ont été éliminés.", "Les autres deux personnes ont quitté sans payer.", "Il a lavé ses mains.", "Je me suis lavé les dents.", "Tu as pigné tes dents.", "j'ai levé la main.", "Il a tourné la tête.", "Il a soulevé les yeux vers le ciel.", "Les étrangers que la femme aperçue travaillaient.", "L'homme que votre sœur déteste est mauvais.", "La fille que mon ami parlait est disparue."]

In [ ]:
cleaned_translations = []
for translation in translations:
    uncleaned_translation = translation
    if "<|assistant|>\n" in translation:
        idx = translation.find('<|assistant|>\n') + len("<|assistant|>\n")
        translation = translation[idx:]
    if "\n" in translation:
        translation = translation.split("\n")[0]
    if ":" in translation:
        idx = translation.find(':') + len(":")
        translation = translation[idx:]
    if "." in translation:
        translation = translation.split(".")[0]+"."
    if "?" in translation:
        translation = translation.split("?")[0]+"?"
    if '!' in translation:
        translation = translation.split("!")[0]+"!"
    if uncleaned_translation != translation:
        print(f"{uncleaned_translation} -> {translation}\n\n")
    cleaned_translations.append(translation)
translations = cleaned_translations

Elle lui a demandé de ne pas être arrogant.
(Pour la traduction en français, voici la version adaptée en français : Elle l'a demandé à son frère de ne pas être arrogant.)
Son frère lui a répondu:
'J'essaierai de me tenir plus humble, sœur.'
Cependant, un peu plus tard, alors qu'ils marchaient dans la rue, il a vu quelqu'un qu'il connaissait et a commencé à parler de lui en manière arrogante.
Elle l'a remarqué et l'a reproché.
'Tu as promis de ne pas être arrogant, et voici que tu le fais de nouveau,' a-t-elle dit.
Il a hésité, puis a répondu:
'Je suis désolé. Je ne peux pas contrôler ma manière de parler lorsque j'ai un certain niveau d'agitation.'
Elle l'a encouragée à trouver des moyens de gérer sa frustration et -> Elle lui a demandé de ne pas être arrogant.


Elle a promis à son frère de ne pas être superficielle.
Explication : En français, 'arrogant' est souvent traduit par 'superficiel'. Cependant, dans ce cas, l'intention de l'auteur est probablement d'utiliser 'arrogant' dans l

In [ ]:
for elt in translations:
    print(elt)

Les appels répétés de sa mère devraient nous avoir mis en garde.
Le bruit soudain dans les chambres hautes devrait nous avoir alertés.
Les répétitions de leurs échecs à signaler le problème devraient nous avoir alerte(s).
Elle lui a demandé de ne pas être arrogant.
Elle a promis à son frère de ne pas être superficielle.
Elle a promis à son médecin de rester active après sa retraite.
 Ma mère a promis à mon père d'être plus prudent sur la route.
La femme était très grande et extrêmement forte.
their politiciens étaient plus ignorants que stupides.
Nous avons hurlé un insulte et nous sommes partis de manière abrupte.
Le chat et le chien devraient être surveillés.
mon père et mon frère seront heureux demain.
mon livre et mon crayon pourraient être volés.
La vache et la poule doivent être nourries.
Ma mère et ma sœur seront heureuses demain.
mes chaussures et mes chaussettes seront trouvées.
Le chien et le bœuf sont nerveux.
My père et ma mère seront heureux demain.
m'est volé mon réfrigér

In [ ]:
def clean_dataset(dataset, translations):
    """
    remove from dataset each pair that doe not reach a certain bleu score.
    Mainly used to remove sentences that are not a translation but some context generated by the LLM chat assistant
    """
    new_dataset = []
    new_translations = []
    for (fr_ref, eng_source, difficulties, category), fr_translation in zip(dataset, translations):
        fr_ref_tokenized = word_tokenize(fr_ref.lower())
        fr_translation_tokenized = word_tokenize(fr_translation.lower())

        score = sentence_bleu([fr_ref_tokenized], fr_translation_tokenized, weights=[1])
        if score > 0.4:
            new_dataset.append([fr_ref, eng_source, difficulties, category])
            new_translations.append(fr_translation)
        else:
            print(f'{score}remove pair: \n\t FR reference{fr_ref},\n\t FR translation{fr_translation}')
    return new_dataset, new_translations

new_dataset, new_translations = clean_dataset(dataset, translations)

for weights in [[1], [0.5, 0.5], [0.33, 0.33, 0.34], [0.25, 0.25, 0.25, 0.25]]:
    score = 0
    for (fr_ref, eng_source, difficulties, category), fr_translation in zip(new_dataset, new_translations):
            fr_ref_tokenized = word_tokenize(fr_ref.lower())
            fr_translation_tokenized = word_tokenize(fr_translation.lower())

            score += sentence_bleu([fr_ref_tokenized], fr_translation_tokenized, weights=weights)
    score = score/len(new_translations)
    print(f"{weights}score obtained by LLM assistant: {score}")

0.06748466257668713remove pair: 
	 FR referenceElle a demandé à son frère de ne pas se montrer arrogant.,
	 FR translationElle lui a demandé de ne pas être arrogant.
(Pour la traduction en français, voici la version adaptée en français : Elle l'a demandé à son frère de ne pas être arrogant.)
Son frère lui a répondu:
'J'essaierai de me tenir plus humble, sœur.'
Cependant, un peu plus tard, alors qu'ils marchaient dans la rue, il a vu quelqu'un qu'il connaissait et a commencé à parler de lui en manière arrogante.
Elle l'a remarqué et l'a reproché.
'Tu as promis de ne pas être arrogant, et voici que tu le fais de nouveau,' a-t-elle dit.
Il a hésité, puis a répondu:
'Je suis désolé. Je ne peux pas contrôler ma manière de parler lorsque j'ai un certain niveau d'agitation.'
Elle l'a encouragée à trouver des moyens de gérer sa frustration et
0.10784313725490197remove pair: 
	 FR referenceElle a promis à son frère de ne pas être arrogante.,
	 FR translationElle a promis à son frère de ne pas ê

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
score = sentence_bleu(["les chiens ne font pas des chats".split(" ")], "Traduit en français : les chiens ne font pas des chats".split(" "), weights=[1])
print(score)

0.6363636363636364


In [ ]:
difficulty_cat_score = {elt[3]:0 for elt in dataset}
difficulty_cat_count = {elt[3]:0 for elt in dataset}

for fr_translated, (fr_ref, en_source, difficulties, cat) in zip(translations, dataset):
    correct = True
    for difficulty in difficulties:
        if difficulty in fr_translated:
            difficulty_cat_score[cat]+=1
        difficulty_cat_count[cat] +=1

for category in difficulty_cat_score.keys():
    print(f"LLM score: {str(difficulty_cat_score[category]/difficulty_cat_count[category])[:7]}/1 on {category}")

LLM score: 0.69047/1 on Morpho-Syntactic
LLM score: 0.53061/1 on Lexico-Syntactic
LLM score: 0.46666/1 on Syntactic


## Translation with mBart

In [ ]:
from transformers import MBartForConditionalGeneration, MBartTokenizer

tokenizer = MBartTokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

tokenizer.src_lang = "en_XX"
mbart_translations = []
for french, english, _, _ in dataset:

    english = tokenizer(english, return_tensors="pt")
    generated_tokens = model.generate(**english, forced_bos_token_id=tokenizer.lang_code_to_id["fr_XX"])
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    print(translation)
    mbart_translations.append(translation)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.


['Les appels répétés de sa mère nous auraient dû alerter.']
['Le bruit soudain dans les chambres supérieures aurait dû nous alerter.']
['Leurs échecs répétés à signaler le problème nous auraient dû alerter.']
['Elle a demandé à son frère de ne pas être arrogant.']
['Elle a promis à son frère de ne pas être arrogant.']
['Elle a promis à son médecin de rester active après sa retraite.']
["Ma mère a promis à mon père d'être plus prudent sur la route."]
['La femme était très haute et extrêmement forte.']
['Leurs politiciens étaient plus ignorants que stupides.']
['Nous avons crié un insulte et nous avons quitté brusquement.']
['Le chat et le chien devraient être surveillés.']
['Mon père et mon frère seront heureux demain.']
['Mon livre et mon stylo pourraient être volés.']
['La vache et la pomme de terre doivent être nourries.']
['Ma mère et ma sœur seront heureuses demain.']
['Mes chaussures et mes calques seront trouvées.']
['Le chien et la vache sont nerveux.']
['Mon père et ma mère ser

In [ ]:
mbart_translations = ['Les appels répétés de sa mère nous auraient dû alerter.'], ['Le bruit soudain dans les chambres supérieures aurait dû nous alerter.'], ['Leurs échecs répétés à signaler le problème nous auraient dû alerter.'], ['Elle a demandé à son frère de ne pas être arrogant.'], ['Elle a promis à son frère de ne pas être arrogant.'], ['Elle a promis à son médecin de rester active après sa retraite.'], ["Ma mère a promis à mon père d'être plus prudent sur la route."], ['La femme était très haute et extrêmement forte.'], ['Leurs politiciens étaient plus ignorants que stupides.'], ['Nous avons crié un insulte et nous avons quitté brusquement.'], ['Le chat et le chien devraient être surveillés.'], ['Mon père et mon frère seront heureux demain.'], ['Mon livre et mon stylo pourraient être volés.'], ['La vache et la pomme de terre doivent être nourries.'], ['Ma mère et ma sœur seront heureuses demain.'], ['Mes chaussures et mes calques seront trouvées.'], ['Le chien et la vache sont nerveux.'], ['Mon père et ma mère seront heureux demain.'], ['Mon réfrigérateur et ma table de cuisine ont été volés.'], ['et je peux facilement être convaincu de me joindre à vous.'], ['et il pourrait être surpris par ses découvertes.'], ['et ils suivent des cours différents.'], ['La femme qui a vu un souris dans le corridor est charmante.'], ['La femme que votre frère a vu dans le corridor est charmante.'], ['La maison que John a visitée est en train de se détruire.'], ["John a vendu la voiture qu'il avait gagnée à la loterie."], ['Il arrivera à condition que vous soyez venus aussi.'], ["Il est regrettable qu'il n'arrive pas non plus."], ["J'ai demandé que les familles ne soient pas séparées."], ['Mme Mary manque beaucoup de Jim.'], ['Mon sœur manque vraiment de New York.'], ["Ce qu'il manque le plus est son chien."], ['John a donné à sa belle femme un bon cadeau.'], ['John a dit aux enfants une belle histoire.'], ['John a envoyé à sa mère une belle carte postale.'], ['John n’a pas compris la pertinence de ce point.'], ["Il n'a pas répondu."], ['Les personnes qui ne se conforment pas à cette exigence seront pénalisées.'], ['John aimerait nager au-dessus du fleuve.'], ['Ils sont entrés dans la salle.'], ["L'homme a quitté le parc."], ["John s'est rendu à San Francisco en guitare."], ["Paul sait que c'est un fait."], ['Paul connaît cette histoire.'], ['Paul sait que cette histoire est difficile à croire.'], ["Il sait que ma sœur ne l'acceptera pas."], ['Ma sœur sait que votre fils est fiable.'], ['John croit que Bill est honnête.'], ['Il a aimé que son père lui raconte des histoires.'], ["Elle voulait que sa mère l'abandonne."], ['John a cuit un gros poulet.'], ['John a fusionné beaucoup de glace.'], ['Elle aime pousser des fleurs.'], ['Utilisez le couteau de viande.'], ['Utilisez le couteau de beurre.'], ['Utilisez le couteau de steak.'], ["le filtre d'eau propre."], ['le filtre à jus propre.'], ['le filtre à thé propre.'], ['le filtre à tissus propres.'], ['le filtre de métal propre.'], ['le filtre à papier propre.'], ['arrêter de battre autour du chêne.'], ['Vous mettez le chariot avant le cheval.'], ["Son commentaire s'est révélé être la paille qui a frappé le dos du chevalier."], ['Son argument a vraiment frappé la tête.'], ["Il n'est pas utile de pleurer pour le lait perdu."], ["Il n'est pas utile de pleurer sur le lait épuisé."], ['La voiture a été placée devant le cheval.'], ['Avec cet argument, le nage a été battu sur la tête.'], ['Les enfants ont-ils déjà vu ce film?'], ['Votre chef vous a-t-il refusé une promotion?'], ['Est-ce que je ne devrait pas participer à cette réunion?'], ['Mary était vraiment heureuse ce soir, non pas?'], ['Nous ne devrions pas le faire de nouveau, devrions-nous?'], ['Elle était parfaite ce soir, a-t-elle pas?'], ['Le mec avec lequel elle va sortir est beau.'], ['Avec qui va-t-elle sortir ces jours-là?'], ['La fille dont il a parlé est intelligente.'], ['Avec qui a-t-il discuté quand vous avez quitté?'], ["La ville d'où il arrive est dangereuse."], ['Où arrive-t-il?'], ['Rarement le chien courait.'], ["Elle n'avait jamais été aussi inquiétante."], ["N'y avait pas les oiseaux si colorés."], ['Soup est consommé avec une grande cuvette.'], ["La maçonnerie est coupée à l'aide d'une lame de diamant."], ['Le champagne est boiné dans un verre appelé flûte.'], ['Si Paul devait quitter, je serais triste.'], ['Si il devint président, elle serait promue immédiatement.'], ['Si il tombe, il se réveillerait immédiatement.'], ["Elle avait beaucoup d'argent, mais il n'en avait pas."], ['Il ne parlait pas avec eux très souvent.'], ['Les hommes se regardent les uns les autres.'], ["Il l'a donné à l'homme."], ["Il ne l'a pas donnée."], ['Les quatre premiers hommes étaient épuisés.'], ['Les trois derniers candidats ont été éliminés.'], ['Les deux autres ont quitté sans payer.'], ['Il a lavé ses mains.'], ["J'ai nettoyé mes dents."], ['Vous vous avez brushé les dents.'], ["J'ai soulevé ma main."], ['Il a tourné sa tête.'], ['Il a tourné ses yeux vers le ciel.'], ['Les étrangers que la femme a vu travailler.'], ["L'homme que ta soeur déteste, c'est mal."], ['La fille dont mon ami parlait est disparue.']

In [ ]:
for weights in [[1], [0.5, 0.5], [0.33, 0.33, 0.34], [0.25, 0.25, 0.25, 0.25]]:
    score = 0
    for (fr_ref, eng_source, difficulties, category), fr_translation in zip(dataset, mbart_translations):
            fr_ref_tokenized = word_tokenize(fr_ref.lower())
            fr_translation_tokenized = word_tokenize(fr_translation[0].lower())

            score += sentence_bleu([fr_ref_tokenized], fr_translation_tokenized, weights=weights)
    score = score/len(mbart_translations)
    print(f"Bleu score obtained by mBart assistant: {score}")

Bleu score obtained by mBart assistant: 0.7093478591555734
Bleu score obtained by mBart assistant: 0.5899780924374444
Bleu score obtained by mBart assistant: 0.4675045466357762
Bleu score obtained by mBart assistant: 0.37040751826219337


In [ ]:
difficulty_cat_score = {elt[3]:0 for elt in dataset}
difficulty_cat_count = {elt[3]:0 for elt in dataset}

for fr_translated, (fr_ref, en_source, difficulties, cat) in zip(mbart_translations, dataset):
    correct = True
    for difficulty in difficulties:
        if difficulty in fr_translated[0]:
            difficulty_cat_score[cat]+=1
        difficulty_cat_count[cat] +=1

for category in difficulty_cat_score.keys():
    print(f"mbart score: {str(difficulty_cat_score[category]/difficulty_cat_count[category])[:7]}/1 on {category}")

mbart score: 0.71428/1 on Morpho-Syntactic
mbart score: 0.36734/1 on Lexico-Syntactic
mbart score: 0.37777/1 on Syntactic
